## Код для интерпретации смайликов позволяет увеличить эффективность предсказаний при Named Entity-Oriented Sentiment Analysis

In [6]:
import pandas as pd
import re

In [7]:
df_names = pd.read_pickle('sentiment_texts.pickle')
df_names.head()

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False


In [11]:
emoji = pd.read_excel('emoji_ru.xlsx')
emoji.head()

,Unnamed: 0,Emoji_ID,Emoji,Unicode_Name,Ru_Name,Class,Total_Occurrence,Negativity_Occurrence_N(Negative),Neutrality_Occurrence_N(Neutral),Positivity_Occurrence_N(Positive),Negativite_Probability_P(Negative),Neutral_Probability_P(Neutral),Positive_Probability_P(Positive),Sentiment_Score_S,label
0,1,b'\xf0\x9f\x98\x82',😂,Face With Tears Of Joy,Лицо со слезами радости,Facial Expression,25908,8106,2638,15164,0.312876,0.101822,0.585302,0.272426,2
1,2,b'\xf0\x9f\x92\x94',💔,Broken Heart,Разбитое сердце,Heart,18564,17781,342,441,0.957822,0.018423,0.023756,-0.934066,0


In [9]:
emoji_mapping = dict(zip(emoji['Emoji'], emoji['Ru_Name']))
emoji_pattern = re.compile('|'.join(map(re.escape, emoji_mapping.keys())))

def replace_emoji_names(text):
    def replace(match):
        return " " + emoji_mapping[match.group(0)] + " "

    return emoji_pattern.sub(replace, text)

df_names['MessageText'] = df_names['MessageText'].apply(replace_emoji_names)
remove_space = lambda text: re.sub(r'\s{2,}', ' ', text)
df_names['MessageText'] = df_names['MessageText'].apply(remove_space)

In [10]:
df_names.head()

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,Предупреждение ️ Россия #SELG #дивиденд сд Се...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False


In [12]:
df_names.to_excel('sentiment_texts_with_emojis.xlsx', index=False)